# 用于预训练BERT的数据集

## BERT预训练的两个核心任务

BERT（Bidirectional Encoder Representations from Transformers）预训练包含两个任务：

### 1. 遮蔽语言模型 (MLM - Masked Language Model)
- **目的**：让模型学习双向上下文表示
- **做法**：随机遮蔽输入中15%的词元，让模型预测被遮蔽的词
- **遮蔽策略**：
  - 80% 替换为 `<mask>` 
  - 10% 替换为随机词
  - 10% 保持不变

### 2. 下一句预测 (NSP - Next Sentence Prediction)
- **目的**：让模型理解句子之间的关系
- **做法**：给定句子A和句子B，预测B是否是A的下一句
- **数据构造**：50%是真实的下一句，50%是随机句子

## 数据流程图

```
原始文本 → 分段落 → 生成句子对(NSP) → 遮蔽词元(MLM) → 填充对齐 → DataLoader
```

In [1]:
import os
import random
import torch
from d2l import torch as d2l

In [2]:
def _read_wiki(data_dir):
    """
    读取WikiText-2数据集并预处理成段落格式
    
    WikiText-2数据集结构：
    - 每一行是一个段落
    - 段落内的句子用 " . " 分隔
    
    返回值结构：
    paragraphs = [
        ["sentence1", "sentence2", ...],  # 段落1
        ["sentence1", "sentence2", ...],  # 段落2
        ...
    ]
    """
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    
    paragraphs = []
    for line in lines:
        # 按 " . " 分割成句子列表
        sentences = line.strip().lower().split(' . ')
        # 只保留包含至少2个句子的段落（NSP任务需要句子对）
        if len(sentences) >= 2:
            paragraphs.append(sentences)
    
    # 打乱段落顺序，增加数据随机性
    random.shuffle(paragraphs)
    return paragraphs

## 第一步：读取和预处理数据

WikiText-2是一个从Wikipedia提取的语言建模数据集，包含约200万个词元。

In [3]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    """
    为NSP任务生成句子对
    
    参数：
        sentence: 当前句子（句子A）
        next_sentence: 原本的下一个句子
        paragraphs: 所有段落，用于采样负样本
    
    返回：
        sentence: 句子A
        next_sentence: 句子B（可能是真实下一句或随机句子）
        is_next: 布尔值，True表示是真实的下一句
    
    策略：50%概率返回真实下一句，50%概率返回随机句子
    """
    if random.random() < 0.5:
        # 正样本：使用真实的下一个句子
        is_next = True
    else:
        # 负样本：从随机段落中随机选一个句子
        # random.choice(paragraphs) -> 随机选一个段落
        # random.choice(...) -> 从该段落中随机选一个句子
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

## 第二步：生成下一句预测(NSP)任务数据

NSP任务的目标是判断两个句子是否相邻。训练数据构造方式：
- **正样本(is_next=True)**：取相邻的两个句子
- **负样本(is_next=False)**：取一个句子 + 随机句子

BERT输入格式：`<cls> 句子A <sep> 句子B <sep>`

In [4]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    """
    从一个段落生成所有NSP训练样本
    
    参数：
        paragraph: 当前段落（句子列表，每个句子是词元列表）
        paragraphs: 所有段落（用于采样负样本）
        vocab: 词表（此函数中未使用，但保持接口一致）
        max_len: 最大序列长度
    
    返回：
        nsp_data_from_paragraph: [(tokens, segments, is_next), ...]
        - tokens: ['<cls>', 词1, 词2, ..., '<sep>', 词1, 词2, ..., '<sep>']
        - segments: [0, 0, ..., 0, 1, 1, ..., 1] 表示属于句子A还是句子B
        - is_next: True/False
    """
    nsp_data_from_paragraph = []
    
    # 遍历段落中的相邻句子对
    for i in range(len(paragraph) - 1):
        # 生成句子对（可能是正样本或负样本）
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        
        # 检查长度：tokens_a + tokens_b + 3个特殊词元(<cls>, <sep>, <sep>)
        # 如果超过max_len则跳过这个样本
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        
        # 使用d2l工具函数构造BERT输入格式
        # tokens: ['<cls>'] + tokens_a + ['<sep>'] + tokens_b + ['<sep>']
        # segments: [0]*len(句子A部分) + [1]*len(句子B部分)
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    
    return nsp_data_from_paragraph

In [5]:
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    """
    对tokens进行遮蔽处理，生成MLM任务的输入和标签
    
    参数：
        tokens: 原始词元列表 ['<cls>', 'hello', 'world', '<sep>', ...]
        candidate_pred_positions: 可以被遮蔽的位置索引列表（不包括特殊词元）
        num_mlm_preds: 需要遮蔽的词元数量（约15%）
        vocab: 词表（用于随机替换时采样）
    
    返回：
        mlm_input_tokens: 遮蔽后的词元列表（用于模型输入）
        pred_positions_and_labels: [(位置, 原始词元), ...] 用于计算损失
    """
    # 复制一份tokens，避免修改原始数据
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    
    # 随机打乱候选位置，然后选取前num_mlm_preds个
    random.shuffle(candidate_pred_positions)
    
    for mlm_pred_position in candidate_pred_positions:
        # 已选够需要预测的数量，退出循环
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        
        masked_token = None
        rand = random.random()
        
        if rand < 0.8:
            # 80%概率：替换为<mask>词元
            masked_token = '<mask>'
        elif rand < 0.9:
            # 10%概率：保持原词不变（但仍然要预测）
            masked_token = tokens[mlm_pred_position]
        else:
            # 10%概率：替换为词表中的随机词
            masked_token = random.choice(vocab.idx_to_token)
        
        # 在输入中进行替换
        mlm_input_tokens[mlm_pred_position] = masked_token
        # 记录位置和原始词元（作为预测标签）
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    
    return mlm_input_tokens, pred_positions_and_labels

## 第三步：生成遮蔽语言模型(MLM)任务数据

MLM任务的核心思想是"完形填空"：随机遮蔽一些词，让模型根据上下文预测被遮蔽的词。

### 遮蔽策略（针对被选中的15%词元）：
| 概率 | 处理方式 | 原因 |
|------|----------|------|
| 80% | 替换为`<mask>` | 主要的遮蔽方式 |
| 10% | 保持不变 | 避免模型认为`<mask>`一定要预测 |
| 10% | 替换为随机词 | 增加鲁棒性 |

这种策略可以避免预训练和微调之间的不匹配问题。

In [6]:
def _get_mlm_data_from_tokens(tokens, vocab):
    """
    从词元序列生成MLM任务的完整数据
    
    参数：
        tokens: 词元列表 ['<cls>', 'the', 'cat', '<sep>', 'sat', '<sep>']
        vocab: 词表对象
    
    返回：
        token_ids: 遮蔽后的词元ID列表（模型输入）
        pred_positions: 被遮蔽的位置列表
        mlm_pred_label_ids: 被遮蔽位置的原始词元ID（预测标签）
    
    示例：
        输入tokens: ['<cls>', 'the', 'cat', 'sat', '<sep>']
        假设遮蔽位置2(cat)和3(sat)
        输出:
            token_ids: [cls_id, the_id, mask_id, random_id, sep_id]
            pred_positions: [2, 3]
            mlm_pred_label_ids: [cat_id, sat_id]
    """
    # 找出所有可以被遮蔽的位置（排除特殊词元）
    candidate_pred_positions = []
    for i, token in enumerate(tokens):
        # <cls>和<sep>是特殊词元，不参与MLM预测
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    
    # 计算需要遮蔽的词元数量：约15%，但至少1个
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    
    # 执行遮蔽操作
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    
    # 按位置排序，保证顺序一致性
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    
    # 分离位置和标签
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    
    # 将词元转换为ID（vocab[tokens]会调用vocab的__getitem__方法）
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

In [7]:
def _pad_bert_inputs(examples, max_len, vocab):
    """
    将所有样本填充到相同长度，转换为PyTorch张量
    
    参数：
        examples: [(token_ids, pred_positions, mlm_labels, segments, is_next), ...]
        max_len: 最大序列长度
        vocab: 词表对象
    
    返回（7个列表，每个列表包含所有样本的对应数据）：
        all_token_ids: 填充后的输入序列 [batch, max_len]
        all_segments: 段落标识 [batch, max_len]
        valid_lens: 有效长度（不含<pad>）[batch]
        all_pred_positions: MLM预测位置 [batch, max_num_mlm_preds]
        all_mlm_weights: MLM损失权重 [batch, max_num_mlm_preds]
        all_mlm_labels: MLM预测标签 [batch, max_num_mlm_preds]
        nsp_labels: NSP标签 [batch]
    """
    # MLM预测的最大数量 = 序列长度 * 15%
    max_num_mlm_preds = round(max_len * 0.15)
    
    # 初始化存储列表
    all_token_ids, all_segments, valid_lens = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    
    for (token_ids, pred_positions, mlm_pred_label_ids, segments, is_next) in examples:
        # ===== 1. 填充token_ids到max_len =====
        # 在序列末尾添加<pad>词元
        padding_len = max_len - len(token_ids)
        all_token_ids.append(torch.tensor(
            token_ids + [vocab['<pad>']] * padding_len, 
            dtype=torch.long))
        
        # ===== 2. 填充segments到max_len =====
        # 填充部分的segment标识为0
        all_segments.append(torch.tensor(
            segments + [0] * padding_len, 
            dtype=torch.long))
        
        # ===== 3. 记录有效长度（用于attention mask）=====
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        
        # ===== 4. 填充pred_positions到max_num_mlm_preds =====
        mlm_padding_len = max_num_mlm_preds - len(pred_positions)
        all_pred_positions.append(torch.tensor(
            pred_positions + [0] * mlm_padding_len, 
            dtype=torch.long))
        
        # ===== 5. 设置mlm_weights（真实预测位置权重为1，填充位置权重为0）=====
        # 这样在计算损失时，填充位置的损失会被过滤掉
        all_mlm_weights.append(torch.tensor(
            [1.0] * len(mlm_pred_label_ids) + [0.0] * mlm_padding_len,
            dtype=torch.float32))
        
        # ===== 6. 填充mlm_labels =====
        all_mlm_labels.append(torch.tensor(
            mlm_pred_label_ids + [0] * mlm_padding_len, 
            dtype=torch.long))
        
        # ===== 7. NSP标签（True->1, False->0）=====
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

## 第四步：填充数据到固定长度

由于不同样本的序列长度不同，需要将它们填充到相同长度才能批量处理。

### 需要填充的内容：
| 数据 | 填充值 | 说明 |
|------|--------|------|
| token_ids | `<pad>` | 输入序列填充 |
| segments | 0 | 段落标识填充 |
| pred_positions | 0 | MLM预测位置填充 |
| mlm_weights | 0.0 | 填充位置的损失权重为0 |
| mlm_labels | 0 | MLM标签填充 |

In [8]:
class _WikiTextDataset(torch.utils.data.Dataset):
    """
    WikiText-2 BERT预训练数据集
    
    数据处理流程：
    1. 分词：将句子字符串转换为词元列表
    2. 构建词表：基于所有句子构建词表
    3. 生成NSP数据：为每个段落生成句子对
    4. 生成MLM数据：对每个句子对进行遮蔽处理
    5. 填充对齐：将所有样本填充到相同长度
    """
    
    def __init__(self, paragraphs, max_len):
        """
        参数：
            paragraphs: 段落列表，每个段落是句子字符串列表
                       [["sentence1", "sentence2"], ["sentence3", "sentence4"], ...]
            max_len: 最大序列长度
        """
        # ===== 步骤1: 分词 =====
        # 将每个句子从字符串转换为词元列表
        # 输入: [["hello world", "foo bar"], ...]
        # 输出: [[["hello", "world"], ["foo", "bar"]], ...]
        paragraphs = [d2l.tokenize(paragraph, token='word') 
                      for paragraph in paragraphs]
        
        # ===== 步骤2: 构建词表 =====
        # 将所有句子展平成一个列表用于构建词表
        sentences = [sentence for paragraph in paragraphs 
                     for sentence in paragraph]
        # min_freq=5: 词频少于5的词会被替换为<unk>
        # reserved_tokens: 预留的特殊词元
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>',   # 填充词元
            '<mask>',  # MLM遮蔽词元
            '<cls>',   # 句子开头词元
            '<sep>'    # 句子分隔词元
        ])
        
        # ===== 步骤3: 生成NSP数据 =====
        # 为每个段落生成所有可能的句子对
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        
        # ===== 步骤4: 生成MLM数据 =====
        # 对每个句子对进行遮蔽处理
        # 输入: (tokens, segments, is_next)
        # 输出: (token_ids, pred_positions, mlm_labels, segments, is_next)
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                     + (segments, is_next))
                    for tokens, segments, is_next in examples]
        
        # ===== 步骤5: 填充到固定长度 =====
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        """返回第idx个样本的所有数据"""
        return (self.all_token_ids[idx],      # 输入token IDs
                self.all_segments[idx],        # 段落标识(0或1)
                self.valid_lens[idx],          # 有效长度
                self.all_pred_positions[idx],  # MLM预测位置
                self.all_mlm_weights[idx],     # MLM损失权重
                self.all_mlm_labels[idx],      # MLM预测标签
                self.nsp_labels[idx])          # NSP标签(0或1)

    def __len__(self):
        """返回数据集大小"""
        return len(self.all_token_ids)

## 第五步：创建PyTorch Dataset

将所有数据处理步骤整合到一个Dataset类中，便于与DataLoader配合使用。

In [9]:
def load_data_wiki(batch_size, max_len):
    """
    加载WikiText-2数据集用于BERT预训练
    
    参数：
        batch_size: 批量大小
        max_len: 最大序列长度（包含特殊词元）
    
    返回：
        train_iter: 训练数据迭代器
        vocab: 词表对象
    """
    # 使用本地已解压的数据目录
    data_dir = '../data/wikitext-2'
    
    # 读取并预处理数据
    paragraphs = _read_wiki(data_dir)
    
    # 创建数据集
    train_set = _WikiTextDataset(paragraphs, max_len)
    
    # 创建数据加载器
    # 注意：在Jupyter notebook中需要设置num_workers=0
    # 因为notebook中定义的类无法被pickle序列化用于多进程
    train_iter = torch.utils.data.DataLoader(
        train_set, 
        batch_size=batch_size,
        shuffle=True,
        num_workers=0  # notebook环境下必须为0
    )
    
    return train_iter, train_set.vocab

## 第六步：数据加载器

将数据集包装成DataLoader，支持批量加载和多进程预取。

In [10]:
# 设置超参数
batch_size = 512  # 批量大小
max_len = 64      # 最大序列长度

# 加载数据
train_iter, vocab = load_data_wiki(batch_size, max_len)

# 查看第一个batch的形状
for (tokens_X, segments_X, valid_lens_x, pred_positions_X, 
     mlm_weights_X, mlm_Y, nsp_y) in train_iter:
    
    print(f"tokens_X:        {tokens_X.shape}")        # [512, 64] - 输入序列
    print(f"segments_X:      {segments_X.shape}")      # [512, 64] - 段落标识
    print(f"valid_lens_x:    {valid_lens_x.shape}")    # [512] - 有效长度
    print(f"pred_positions_X:{pred_positions_X.shape}")# [512, 10] - MLM位置 (64*0.15≈10)
    print(f"mlm_weights_X:   {mlm_weights_X.shape}")   # [512, 10] - MLM权重
    print(f"mlm_Y:           {mlm_Y.shape}")           # [512, 10] - MLM标签
    print(f"nsp_y:           {nsp_y.shape}")           # [512] - NSP标签
    break

tokens_X:        torch.Size([512, 64])
segments_X:      torch.Size([512, 64])
valid_lens_x:    torch.Size([512])
pred_positions_X:torch.Size([512, 10])
mlm_weights_X:   torch.Size([512, 10])
mlm_Y:           torch.Size([512, 10])
nsp_y:           torch.Size([512])


## 测试数据加载

验证数据加载器是否正常工作，并查看各张量的形状。

### 输出形状说明：
| 张量 | 形状 | 说明 |
|------|------|------|
| tokens_X | [batch, max_len] | 输入token IDs |
| segments_X | [batch, max_len] | 段落标识 |
| valid_lens_x | [batch] | 每个样本的有效长度 |
| pred_positions_X | [batch, max_mlm_preds] | MLM预测位置 |
| mlm_weights_X | [batch, max_mlm_preds] | MLM损失权重 |
| mlm_Y | [batch, max_mlm_preds] | MLM预测标签 |
| nsp_y | [batch] | NSP标签 |

In [11]:
# 查看词表大小
print(f"词表大小: {len(vocab)}")

词表大小: 20066
